In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

# 思路
有user_df和没有user_df分别训练一个nn

In [2]:
# !pip install -U deepmatch
!pip freeze | grep deepmatch

deepmatch==0.1.3


In [3]:
import pandas as pd
from deepctr.inputs import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss

/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/luoyonggui/anaconda3/envs/tf14

## preprocess

In [4]:
import random
import numpy as np
from tqdm import tqdm
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

### gen_data_set

In [5]:
def gen_data_set(data, negsample: int=0):
    """
    negsample: 负样本数相对于正样本数的倍数
    """
    data.sort_values("time", inplace=True)
    item_ids = data['item_id'].unique()

    train_set = []
    test_set = []
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['item_id'].tolist()  # 用户看过的电影列表
#         rating_list = hist['rating'].tolist()  # 用户看过的电影评分列表

        if negsample > 0:
            # 产生负样本，策略：从用户没有评分的items中随机有放回采样
            candidate_set = list(set(item_ids) - set(pos_list))
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            if i != len(pos_list) - 1:
                # (user_id, items_list, next_item, label, items_list_length, next_item_rating)
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1,len(hist[::-1]),item_profile.loc[hist[-1]].values.tolist()))
                for negi in range(negsample):
                    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1]),item_profile.loc[hist[-1]].values.tolist()))
            else:
                test_set.append((reviewerID, hist[::-1], pos_list[i],1,len(hist[::-1]),item_profile.loc[hist[-1]].values.tolist()))

    random.shuffle(train_set)
    random.shuffle(test_set)

    print(len(train_set[0]),len(test_set[0]))

    return train_set,test_set

### gen_model_input

In [6]:
def gen_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])
    item_feat_last = np.array([line[5] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {"user_id": train_uid, "item_id": train_iid, "hist_item_id": train_seq_pad,
                         "hist_len": train_hist_len, 'item_feat_last': item_feat_last}

    for key in ["user_gender", "user_age_level", "user_city_level"]:
        train_model_input[key] = user_profile.loc[train_model_input['user_id']][key].values

    return train_model_input,train_label

In [7]:
from code.eda import * 
from code.config import args

In [8]:
# export

whole_click_train, whole_click_val, click_test, test_qtime = load_whole_click_data(args.now_phase, args.DATA_DIR)

# 完整的click数据
whole_click = whole_click_train.append(whole_click_val)

whole_click.shape

2020-05-21 20:15:46.674 | INFO     | code.eda:load_whole_click_data:54 - phase: 0
2020-05-21 20:15:47.099 | INFO     | code.eda:load_whole_click_data:54 - phase: 1
2020-05-21 20:15:47.532 | INFO     | code.eda:load_whole_click_data:54 - phase: 2
2020-05-21 20:15:47.997 | INFO     | code.eda:load_whole_click_data:54 - phase: 3
2020-05-21 20:15:48.540 | INFO     | code.eda:load_whole_click_data:54 - phase: 4
2020-05-21 20:15:49.099 | INFO     | code.eda:load_whole_click_data:54 - phase: 5
2020-05-21 20:15:49.690 | INFO     | code.eda:load_whole_click_data:54 - phase: 6
2020-05-21 20:15:59.886 | INFO     | code.eda:load_whole_click_data:84 - whole_click_train: (954588, 4), whole_click_val: (12081, 4), click_test: (175274, 4), test_qtime: (12081, 3)


(966669, 4)

In [9]:
user_df = get_user_data()

In [10]:
item_df = get_item_data()

/Users/luoyonggui/PycharmProjects/myappsexlib/a_0330_0527_Modern_E_Commerce_Platform_Debiasing/code/eda.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  converters={'item_id':str})


In [11]:
user_df.columns

Index(['user_id', 'user_age_level', 'user_gender', 'user_city_level'], dtype='object')

In [12]:
data = pd.merge(pd.merge(whole_click, user_df), item_df, how='left')
# data = pd.merge(pd.merge(click_test.iloc[:10000], user_df, how='left'), item_df, how='left')

In [15]:
data.shape

(255162, 263)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255162 entries, 0 to 255161
Columns: 263 entries, user_id to 255
dtypes: float64(256), int64(1), object(6)
memory usage: 513.9+ MB


In [13]:
data = data.fillna(0)

In [18]:
data.head(2)

,user_id,item_id,time,phase,user_age_level,user_gender,user_city_level,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,12136,28195,0.9837396808253338,0,7,F,4,4.158704,-2.213841,3.299623,-3.816672,1.817951,-0.562453,-3.637601,-0.311089,-5.418423,-2.103671,-0.442526,-3.839618,4.380653,-3.727894,1.042084,-2.424195,3.287570,-0.606853,-1.033391,0.160277,-0.038618,-1.071147,0.780368,-0.15989,-1.548846,-2.328499,-0.205781,-3.355978,-5.415924,-1.503188,-5.305884,-1.843716,-0.245827,3.008088,3.399326,-3.194445,-3.39710,-0.312699,2.398918,4.553303,-2.690272,1.037717,1.235074,...,-1.572505,-1.981386,-1.908951,3.034426,-4.374824,0.489178,-3.051432,-2.832758,-2.986957,-1.558040,4.542650,-0.078699,2.846123,2.866818,-2.151008,0.465624,3.206414,-2.886617,-0.973224,4.858570,0.401090,1.429223,-1.306388,2.436626,0.375304,0.796462,-1.397474,0.019915,-4.959344,-0.943585,0.348210,-2.54955,1.375585,-3.331098,0.260209,-1.60476,4.503458,0.519022,2.495067,-2.482817,-2.561544,-0.005109,-0.008450,-0.523985,1.464158,0.655260,-0.055013,0.743994,2.011905,0.713488
1,12136,8180,0.9837398562121692,0,7,F,4,1.256375,-0.135355,-0.094411,-2.881465,-2.859680,0.219944,-2.651040,-0.448493,-4.705667,-2.737536,-2.651793,-1.025328,3.507258,-0.903140,-0.954022,-0.176986,-0.129408,-0.903812,-1.485689,-1.232078,1.635235,0.191120,2.317544,-0.78316,-0.922034,-0.601006,1.982776,0.541685,-0.317295,1.846201,0.336760,-1.875458,1.001356,-2.466253,1.808531,-1.687020,1.17516,-2.430258,0.835395,3.409604,-2.234955,1.543343,-0.720782,...,2.292294,0.804790,1.512545,-2.500110,-0.781102,-4.062862,-2.441689,1.630657,1.479393,-2.077727,2.233886,-1.078675,0.713491,-0.945299,-0.759625,0.223324,3.597754,-1.059106,1.967239,3.818967,1.521663,-1.366154,1.490747,1.029143,1.499049,-0.089437,-1.194861,0.879334,0.018344,-0.082374,-3.172344,-2.91691,0.775857,3.820774,-1.552581,-3.47731,-1.286526,-1.216785,-0.149395,-1.064123,1.278076,1.322566,-2.070811,-2.692464,-2.537409,-2.680542,0.139775,2.710567,2.210895,-2.694886


In [14]:
SEQ_LEN = 50
negsample = 0

In [15]:
sparse_features = ["item_id", "user_id",
                    "user_gender", "user_age_level", "user_city_level"]


# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`

features = sparse_features
feature_max_idx = {}
lbe_dict = {}
for feature in features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1
    lbe_dict[feature] = lbe



In [16]:
user_profile = data[["user_id",
                    "user_gender", "user_age_level", "user_city_level"]].drop_duplicates('user_id')

item_profile = data[["item_id"]+list(range(256))].drop_duplicates('item_id')

user_profile.set_index("user_id", inplace=True)
item_profile.set_index("item_id", inplace=True)

user_item_list = data.groupby("user_id")['item_id'].apply(list)

In [22]:
item_profile.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
item_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26220,4.158704,-2.213841,3.299623,-3.816672,1.817951,-0.562453,-3.637601,-0.311089,-5.418423,-2.103671,-0.442526,-3.839618,4.380653,-3.727894,1.042084,-2.424195,3.287570,-0.606853,-1.033391,0.160277,-0.038618,-1.071147,0.780368,-0.15989,-1.548846,-2.328499,-0.205781,-3.355978,-5.415924,-1.503188,-5.305884,-1.843716,-0.245827,3.008088,3.399326,-3.194445,-3.39710,-0.312699,2.398918,4.553303,-2.690272,1.037717,1.235074,4.735878,1.678928,-1.881256,0.291053,1.781230,1.398565,3.934110,...,-1.572505,-1.981386,-1.908951,3.034426,-4.374824,0.489178,-3.051432,-2.832758,-2.986957,-1.558040,4.542650,-0.078699,2.846123,2.866818,-2.151008,0.465624,3.206414,-2.886617,-0.973224,4.858570,0.401090,1.429223,-1.306388,2.436626,0.375304,0.796462,-1.397474,0.019915,-4.959344,-0.943585,0.348210,-2.54955,1.375585,-3.331098,0.260209,-1.60476,4.503458,0.519022,2.495067,-2.482817,-2.561544,-0.005109,-0.008450,-0.523985,1.464158,0.655260,-0.055013,0.743994,2.011905,0.713488
67762,1.256375,-0.135355,-0.094411,-2.881465,-2.859680,0.219944,-2.651040,-0.448493,-4.705667,-2.737536,-2.651793,-1.025328,3.507258,-0.903140,-0.954022,-0.176986,-0.129408,-0.903812,-1.485689,-1.232078,1.635235,0.191120,2.317544,-0.78316,-0.922034,-0.601006,1.982776,0.541685,-0.317295,1.846201,0.336760,-1.875458,1.001356,-2.466253,1.808531,-1.687020,1.17516,-2.430258,0.835395,3.409604,-2.234955,1.543343,-0.720782,4.805836,-0.952767,-3.785604,2.363004,0.964829,1.373647,-1.082907,...,2.292294,0.804790,1.512545,-2.500110,-0.781102,-4.062862,-2.441689,1.630657,1.479393,-2.077727,2.233886,-1.078675,0.713491,-0.945299,-0.759625,0.223324,3.597754,-1.059106,1.967239,3.818967,1.521663,-1.366154,1.490747,1.029143,1.499049,-0.089437,-1.194861,0.879334,0.018344,-0.082374,-3.172344,-2.91691,0.775857,3.820774,-1.552581,-3.47731,-1.286526,-1.216785,-0.149395,-1.064123,1.278076,1.322566,-2.070811,-2.692464,-2.537409,-2.680542,0.139775,2.710567,2.210895,-2.694886


In [23]:
user_item_list.head(2)

user_id
1    [62164, 11558, 81694, 58100, 27620, 44657, 503...
2       [53232, 52524, 19348, 22217, 879, 52537, 8197]
Name: item_id, dtype: object

In [24]:
data.head(2)

,user_id,item_id,time,phase,user_age_level,user_gender,user_city_level,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,420,26220,0.9837396808253338,0,8,2,5,4.158704,-2.213841,3.299623,-3.816672,1.817951,-0.562453,-3.637601,-0.311089,-5.418423,-2.103671,-0.442526,-3.839618,4.380653,-3.727894,1.042084,-2.424195,3.287570,-0.606853,-1.033391,0.160277,-0.038618,-1.071147,0.780368,-0.15989,-1.548846,-2.328499,-0.205781,-3.355978,-5.415924,-1.503188,-5.305884,-1.843716,-0.245827,3.008088,3.399326,-3.194445,-3.39710,-0.312699,2.398918,4.553303,-2.690272,1.037717,1.235074,...,-1.572505,-1.981386,-1.908951,3.034426,-4.374824,0.489178,-3.051432,-2.832758,-2.986957,-1.558040,4.542650,-0.078699,2.846123,2.866818,-2.151008,0.465624,3.206414,-2.886617,-0.973224,4.858570,0.401090,1.429223,-1.306388,2.436626,0.375304,0.796462,-1.397474,0.019915,-4.959344,-0.943585,0.348210,-2.54955,1.375585,-3.331098,0.260209,-1.60476,4.503458,0.519022,2.495067,-2.482817,-2.561544,-0.005109,-0.008450,-0.523985,1.464158,0.655260,-0.055013,0.743994,2.011905,0.713488
1,420,67762,0.9837398562121692,0,8,2,5,1.256375,-0.135355,-0.094411,-2.881465,-2.859680,0.219944,-2.651040,-0.448493,-4.705667,-2.737536,-2.651793,-1.025328,3.507258,-0.903140,-0.954022,-0.176986,-0.129408,-0.903812,-1.485689,-1.232078,1.635235,0.191120,2.317544,-0.78316,-0.922034,-0.601006,1.982776,0.541685,-0.317295,1.846201,0.336760,-1.875458,1.001356,-2.466253,1.808531,-1.687020,1.17516,-2.430258,0.835395,3.409604,-2.234955,1.543343,-0.720782,...,2.292294,0.804790,1.512545,-2.500110,-0.781102,-4.062862,-2.441689,1.630657,1.479393,-2.077727,2.233886,-1.078675,0.713491,-0.945299,-0.759625,0.223324,3.597754,-1.059106,1.967239,3.818967,1.521663,-1.366154,1.490747,1.029143,1.499049,-0.089437,-1.194861,0.879334,0.018344,-0.082374,-3.172344,-2.91691,0.775857,3.820774,-1.552581,-3.47731,-1.286526,-1.216785,-0.149395,-1.064123,1.278076,1.322566,-2.070811,-2.692464,-2.537409,-2.680542,0.139775,2.710567,2.210895,-2.694886


In [17]:
train_set, test_set = gen_data_set(data, negsample)

100%|██████████| 6066/6066 [00:41<00:00, 144.44it/s]


6 6


In [26]:
train_set[:1]

[(857,
  [45663, 7999, 43277, 45464, 41444, 25060, 59320],
  4319,
  1,
  7,
  [1.0528055429458618,
   1.128563642501831,
   -0.19476017355918884,
   -2.8232998847961426,
   -1.0138307809829712,
   -0.17025381326675415,
   -1.7379908561706543,
   2.036206722259521,
   -4.2752180099487305,
   -2.896310567855835,
   2.8508329391479488,
   -1.0319020748138428,
   4.0007572174072275,
   0.5457701086997986,
   -1.5669609308242798,
   1.7627053260803225,
   -0.7388931512832642,
   -0.9483394026756288,
   -0.21662521362304688,
   -0.5011007189750671,
   -2.277186870574951,
   -2.8989429473876958,
   0.402845561504364,
   -0.3285502791404724,
   0.6879732608795166,
   -0.11109140515327454,
   4.7324934005737305,
   -1.882719039916992,
   -1.33441424369812,
   3.06593656539917,
   -2.705151319503784,
   -2.0082178115844727,
   -3.4601364135742188,
   -0.5322337150573729,
   2.6357357501983643,
   2.2899339199066158,
   -3.3446762561798096,
   -3.060894012451172,
   2.4056987762451167,
   2.3987

In [18]:
train_seq = [line[1] for line in train_set]

In [28]:
train_seq[:2]

[[45663, 7999, 43277, 45464, 41444, 25060, 59320],
 [19719,
  27928,
  63186,
  63827,
  80452,
  6408,
  54281,
  45307,
  17653,
  24196,
  61131,
  72958,
  9440,
  72479,
  36751,
  51318,
  30367,
  3717,
  74443,
  61311,
  71410,
  23083,
  65564,
  71187,
  52030,
  16487,
  51340,
  45683,
  60834,
  24248,
  17653,
  39669,
  73232,
  66208,
  52686]]

In [19]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

In [30]:
test_model_input

{'user_id': array([5381, 4019, 3664, ..., 1329, 2998, 3026]),
 'item_id': array([15386, 46739, 44077, ...,   923,  3669, 62612]),
 'hist_item_id': array([[27661, 20909, 23085, ...,     0,     0,     0],
        [ 3111, 47064,  3350, ..., 81596, 49674,  7942],
        [20794, 48904, 73892, ...,     0,     0,     0],
        ...,
        [54768, 36653,  3127, ...,     0,     0,     0],
        [39183, 76659, 56312, ..., 27590, 56160, 31816],
        [  701, 76651,  3024, ...,     0,     0,     0]], dtype=int32),
 'hist_len': array([40, 58, 12, ..., 16, 57, 18]),
 'item_feat_last': array([[-1.60878205, -0.08759964, -0.43949628, ..., -0.1404233 ,
          1.99800789, -2.74842024],
        [ 2.21175838, -4.83768463,  3.36969185, ...,  3.06375003,
          2.31905413, -0.42470151],
        [ 2.96954632,  1.35837245,  3.38686228, ...,  1.49628675,
         -1.77107739, -1.30099368],
        ...,
        [ 1.78895044,  1.15014434,  1.34286404, ...,  2.1780057 ,
          1.22753143,  2.76573

In [31]:
test_label

array([1, 1, 1, ..., 1, 1, 1])

In [20]:
embedding_dim = 32

user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("user_gender", feature_max_idx['user_gender'], 16),
                        SparseFeat("user_age_level", feature_max_idx['user_age_level'], 16),
                        SparseFeat("user_city_level", feature_max_idx['user_city_level'], 16),
                        VarLenSparseFeat(SparseFeat('hist_item_id', feature_max_idx['item_id'], embedding_dim,
                                                    embedding_name="item_id"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

item_feature_columns = [SparseFeat('item_id', feature_max_idx['item_id'], embedding_dim)]

In [21]:
# 3.Define Model and train

K.set_learning_phase(True)

model = YoutubeDNN(user_feature_columns, item_feature_columns, num_sampled=200, user_dnn_hidden_units=(128,64, embedding_dim))
# model = MIND(user_feature_columns,item_feature_columns,dynamic_k=True,p=1,k_max=2,num_sampled=5,user_dnn_hidden_units=(64,16),init_std=0.001)

model.compile(optimizer="adam", loss=sampledsoftmaxloss)  # "binary_crossentropy")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [32]:
history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=24, verbose=1, validation_split=0.0, )

Epoch 1/24
243030/243030 [==============================] - 31s 126us/sample - loss: 3.8907
Epoch 2/24
243030/243030 [==============================] - 29s 120us/sample - loss: 3.8230
Epoch 3/24
243030/243030 [==============================] - 29s 118us/sample - loss: 3.7958
Epoch 4/24
243030/243030 [==============================] - 28s 117us/sample - loss: 3.7104
Epoch 5/24
243030/243030 [==============================] - 28s 117us/sample - loss: 3.6789
Epoch 6/24
243030/243030 [==============================] - 29s 118us/sample - loss: 3.6488
Epoch 7/24
243030/243030 [==============================] - 29s 118us/sample - loss: 3.5647
Epoch 8/24
243030/243030 [==============================] - 29s 120us/sample - loss: 3.5314
Epoch 9/24
243030/243030 [==============================] - 30s 122us/sample - loss: 3.5030
Epoch 10/24
243030/243030 [==============================] - 30s 122us/sample - loss: 3.4246
Epoch 11/24
243030/243030 [==============================] - 30s 122us/sample -

In [33]:
test_user_model_input = test_model_input
# all_item_model_input = {"item_id": item_profile['item_id'].values,}
all_item_model_input = {"item_id": np.array(list(item_profile.index)),}

# 以下两行是deepmatch中的通用使用方法，分别获得用户向量模型和物品向量模型
user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

# 输入对应的数据拿到对应的向量
user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(6066, 32)
(82120, 32)


In [34]:
test_user_model_input['user_id'].shape

(6066,)

In [35]:
test_true_label = {line[0]:[line[2]] for line in test_set}  # user_id: [next_item_id]

import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(user_embs, 50)
s = []
hit = 0
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
#     try:
    tt = np.array(list(item_profile.index))
    if True:
#         pred = [item_profile['item_id'].values[x] for x in I[i]]
        pred = [tt[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        if test_true_label[uid] in pred:
            hit += 1
#     except:
# #         print(i)
#         pass
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))

6066it [01:06, 91.33it/s]


recall 0.01071546323771843
hit rate 0.01071546323771843
